# Market Structure Playground

This notebook visualizes the features generated by the `MarketStructureEngine` step-by-step. It's designed to be a visual reference for understanding what each signal looks like and how they might be used.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.dates as mdates
import numpy as np

# Ensure the project root is in the Python path
ROOT = Path().resolve().parents[2]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from finantradealgo.features.market_structure_features import add_market_structure_features
from finantradealgo.market_structure.config import MarketStructureConfig

# --- Configuration ---
DATA_PATH = ROOT / "data" / "ohlcv" / "BTCUSDT_15m.csv"
NUM_BARS = 750

In [ ]:
# Load data and calculate all features
df_source = pd.read_csv(DATA_PATH, parse_dates=["timestamp"])
df = df_source.tail(NUM_BARS).copy()

cfg = MarketStructureConfig()
cfg.swing.lookback = 10
cfg.zone.window_bars = 500

df_features, zones = add_market_structure_features(df, cfg)
df_features = df_features.set_index('timestamp')

print(f"Loaded {len(df_features)} bars.")
print(f"Found {len(zones)} zones.")

## Step 1: Price and Swing Points

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))

ax.plot(df_features.index, df_features['close'], color='k', linewidth=0.9, label='Close')

# Find swing high/low locations
swing_highs = df_features[df_features['ms_swing_high'] == 1]
swing_lows = df_features[df_features['ms_swing_low'] == 1]

ax.scatter(swing_highs.index, swing_highs['high'], marker='^', color='red', s=100, label='Swing High')
ax.scatter(swing_lows.index, swing_lows['low'], marker='v', color='green', s=100, label='Swing Low')

ax.set_title('Price with Swing Highs and Lows')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.5)
plt.show()

## Step 2: Trend Regime Timeline

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

# Price Plot
ax1.plot(df_features.index, df_features['close'], color='k', linewidth=0.9)
ax1.set_title('Price and Trend Regime')
ax1.grid(True, linestyle='--', alpha=0.5)

# Trend Regime Plot
ax2.plot(df_features.index, df_features['ms_trend_regime'], label='Trend Regime')
ax2.fill_between(df_features.index, 0, 1, where=df_features['ms_trend_regime'] == 1, color='green', alpha=0.3, transform=ax2.get_xaxis_transform())
ax2.fill_between(df_features.index, -1, 0, where=df_features['ms_trend_regime'] == -1, color='red', alpha=0.3, transform=ax2.get_xaxis_transform())
ax2.set_yticks([-1, 0, 1])
ax2.set_yticklabels(['Down', 'Range', 'Up'])
ax2.set_title('Trend Regime')
ax2.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Step 3: Fair Value Gaps (FVG)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(df_features.index, df_features['close'], color='k', linewidth=0.9, label='Close')

# Find FVG locations and boundaries
for i in range(2, len(df_features)):
    # Bullish FVG
    if df_features['ms_fvg_up'].iloc[i] == 1:
        fvg_low = df_features['high'].iloc[i-2]
        fvg_high = df_features['low'].iloc[i]
        rect = patches.Rectangle((df_features.index[i-2], fvg_low), df_features.index[i] - df_features.index[i-2], fvg_high - fvg_low, facecolor='green', alpha=0.2)
        ax.add_patch(rect)
    # Bearish FVG
    if df_features['ms_fvg_down'].iloc[i] == 1:
        fvg_high = df_features['low'].iloc[i-2]
        fvg_low = df_features['high'].iloc[i]
        rect = patches.Rectangle((df_features.index[i-2], fvg_low), df_features.index[i] - df_features.index[i-2], fvg_high - fvg_low, facecolor='red', alpha=0.2)
        ax.add_patch(rect)

ax.set_title('Price with Fair Value Gaps (FVG)')
ax.grid(True, linestyle='--', alpha=0.5)
plt.show()

## Step 4: Supply & Demand Zones

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(df_features.index, df_features['close'], color='k', linewidth=0.9, label='Close')

for zone in zones:
    color = 'green' if zone.type == 'demand' else 'red'
    try:
        start_time = df_features.index[zone.first_ts]
        end_time = df_features.index[zone.last_ts]
    except IndexError:
        continue
    
    width = mdates.date2num(end_time) - mdates.date2num(start_time)
    rect = patches.Rectangle(
        (mdates.date2num(start_time), zone.low),
        width,
        zone.high - zone.low,
        linewidth=1,
        edgecolor=color,
        facecolor=color,
        alpha=0.15
    )
    ax.add_patch(rect)

ax.set_title('Price with Supply & Demand Zones')
ax.grid(True, linestyle='--', alpha=0.5)
plt.show()

## Step 5: Break of Structure (BoS) & Change of Character (ChoCh)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(df_features.index, df_features['close'], color='k', linewidth=0.9, label='Close')

# Find event locations
bos_up = df_features[df_features['ms_bos_up'] == 1]
bos_down = df_features[df_features['ms_bos_down'] == 1]
choch = df_features[df_features['ms_choch'] == 1]

ax.scatter(bos_up.index, bos_up['high'], marker='o', facecolors='none', edgecolors='blue', s=150, linewidth=2, label='BoS Up')
ax.scatter(bos_down.index, bos_down['low'], marker='o', facecolors='none', edgecolors='orange', s=150, linewidth=2, label='BoS Down')
ax.scatter(choch.index, choch['close'], marker='x', color='purple', s=150, linewidth=2, label='ChoCh')

ax.set_title('Price with BoS and ChoCh Events')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.5)
plt.show()